In [7]:
x_size = 224
y_size = 224
kind_label = []
cat_img = []

In [8]:
import os
import numpy as np
from tensorflow import keras
import pandas as pd
import glob
from sklearn.model_selection import train_test_split
import matplotlib as mpl
import matplotlib.pyplot as plt
import warnings 
warnings.filterwarnings('ignore')

In [9]:
#識別する品種のリスト
cat_list = ['Abyssinian','Bengal','Birman','Bombay','British_Shorthair','Egyptian_Mau','Maine_Coon','Persian'
            ,'Ragdoll','Russian_Blue','Siamese','Sphynx']


In [14]:
#データセットのロード
for cat_kind in cat_list:
    print(cat_kind)
    file_list = glob.glob('/downloads/images/*.jpg')
    for file in file_list:
        img_path = file
        img = image.load_img(img_path, target_size=(x_size, y_size))
        x = image.img_to_array(img)
        x = preprocess_input(x)
        cat_img.append(x)
        kind_label.append(cat_kind) 



Abyssinian
Bengal
Birman
Bombay
British_Shorthair
Egyptian_Mau
Maine_Coon
Persian
Ragdoll
Russian_Blue
Siamese
Sphynx


In [11]:
#品種ラベルをダミー化
Y_dummy = pd.get_dummies(kind_label)

X_train, X_test, y_train, y_test = train_test_split(
    cat_img, Y_dummy, test_size=0.2, random_state=42)

[]


In [12]:
from keras.applications.inception_v3 import InceptionV3
model = InceptionV3(weights='imagenet')

In [30]:
from keras.models import Model
from keras.layers import Activation, Dense, Flatten
from keras.applications.vgg16 import VGG16
# 中間層を出力するモデル
intermediate_layer_model = Model(inputs=model.input, outputs=model.layers[311].output)

feature = intermediate_layer_model.predict(x)
pd.DataFrame(feature .reshape(-1,1)).plot(figsize=(12, 3))

# Denseレイヤーを接続
x = intermediate_layer_model.output
x = Dense(1024, activation='relu')
predictions = Dense(len(cat_list), activation='softmax')(x)

# 転移学習モデル
transfer_model = Model(inputs=intermediate_layer_model.input, outputs=predictions)

# 一旦全レイヤーをフリーズ
for layer in transfer_model.layers:
    layer.trainable = False

# 最終段のDenseだけ再学習する
transfer_model.layers[312].trainable = True
transfer_model.layers[313].trainable = True

transfer_model.compile(loss='categorical_crossentropy',
                       optimizer='adam',
                       metrics=['accuracy'])

#転移学習
transfer_model.fit(np.array(X_train), np.array(y_train), epochs=10,
                   validation_data=(np.array(X_test), np.array(y_test)))

#精度確認用に出力（必要に応じて）
loss, acc = transfer_model.evaluate(np.array(X_test), np.array(y_test))
print('Loss {}, Accuracy {}'.format(loss, acc))

ValueError: When feeding symbolic tensors to a model, we expect thetensors to have a static batch size. Got tensor with shape: (None, 1024)